# Serving Recommender Models for Online Prediction using NVIDIA Triton and Vertex AI

This notebooks demonstrates serving a HugeCTR model using Triton server on Vertex AI prediction.
The notebook compiles prescriptive guidance for the following tasks:

1. Exporting the Triton ensemble model consisting of NVTabular preprocessing workflow HugeCTR model.
2. Uploading the model and its metadata to Vertex Models.
3. Building a custom container derived from NVIDIA NGC Merlin inference image.
4. Deploy the model to Vertex AI Prediction.
5. Getting the inference on a sample data points using hte endpoint.

## Triton Inference Server Overview

[Triton Inference Server](https://github.com/triton-inference-server/server) provides an inferencing solution optimized for both CPUs and GPUs. Triton can run multiple models from the same or different frameworks concurrently on a single GPU or CPU. In a multi-GPU server, it automatically creates an instance of each model on each GPU to increase utilization without extra coding.It supports real-time inferencing, batch inferencing to maximize GPU/CPU utilization, and streaming inference with built-in support for audio streaming input. It also supports model ensemble for use cases that require multiple models to perform end-to-end inference.

At a high-level, the Triton Inference Server high-level architecture works as follows:
- The model repository is a file-system based repository of the models that Triton will make available for inferencing. 
- Inference requests arrive at the server via either HTTP/REST or gRPC or then routed to the appropriate per-model scheduler. 
- Triton implements multiple scheduling and batching algorithms that can be configured on a model-by-model basis.
- The backend performs inferencing using the inputs provided in the batched requests to produce the requested outputs.

Triton server provides readiness and liveness health endpoints, as well as utilization, throughput, and latency metrics, which enables the integration of Triton into deployment environment, such as Vertex AI Prediction.

In this example, we use Triton to serve an ensemble model that contains data processing workflow and HugeCTR model trained on Criteo data. The model is deployed into Vertex AI Prediction. This is shown in the following figure:

<img src="./images/triton-vertex.png" alt="Triton Architecture" style="width:70%"/>

## Setup

In this section of the notebook you configure your environment settings, including a GCP project, a GCS compute region, and a GCP Bucket. 
You also set the locations of the saved NVTaubular workflow, created in [01-dataset-preprocessing.ipynb](01-dataset-preprocessing.ipynb) and the trained HugeCTR model, created in [02-model-training-hugectr.ipynb](02-model-training-hugectr.ipynb) notebook.

Make sure to update the below cells with the values reflecting your environment.

In [ ]:
import json
import os
from pathlib import Path
import time

from src.serving import export
from src import feature_utils

from google.cloud import aiplatform as vetex_ai

In [ ]:
PROJECT = 'merlin-on-gcp' # Change to your project.
REGION = 'us-central1'  # Change to your region.
BUCKET = 'merlin-on-gcp' # Change to your bucket.

VERSION = 'v01'
ENDPOINT_DISPLAY_NAME = 'criteo-merlin-recommender'
MODEL_DISPLAY_NAME = f'{ENDPOINT_DISPLAY_NAME}-{VERSION}'
WORKSPACE = f'gs://{BUCKET}/{MODEL_DISPLAY_NAME}'
EXPORTED_MODELS_DIR = os.path.join(WORKSPACE, "model_registry")

IMAGE_NAME = 'triton_deploy-hugectr'
IMAGE_URI = f"gcr.io/{PROJECT_ID}/{IMAGE_NAME}"
DOCKERFILE = 'src/Dockerfile.triton'

WORKFLOW_MODEL_DIR = "" # Change to GCS path of the nvt workflow.
HUGECTR_MODEL_DIR = "" # Change to GCS path of the hugectr trained model.

### Initialize Vertex AI SDK

In [ ]:
vertex_ai.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=os.path.join(WORKSPACE, 'stg') 
)

## 1. Exporting the Triton ensemble model

The Triton ensemble model consists of NVTabular preprocessing workflow HugeCTR model

In [ ]:
!gsutil cp -r {WORKFLOW_MODEL_DIR} .

In [ ]:
!gsutil cp -r {HUGECTR_MODEL_DIR} .

### Exporting the ensemble model

In [ ]:
categotical_columns = feature_utils.categotical_columns()
continuous_columns = feature_utils.continuous_columns()
label_columns = feature_utils.label_columns()

local_workflow_path = Path(WORKFLOW_MODEL_DIR).parts[-1]
local_saved_model_path = Path(HUGECTR_MODEL_DIR).parts[-1]
local_exported_ensemble_path = f'triton-ensemble-{time.strftime("%Y%m%d%H%M%S")}'

export.export_ensemble(
    workflow_path=local_workflow_path,
    saved_model_path=local_saved_model_path,
    output_path=local_exported_ensemble_path,
    categotical_columns,
    continuous_columns,
    label_columns)

## 2. Uploading the model and its metadata to Vertex Models.

In [ ]:
!gsutil cp -r {local_exported_ensemble_path} {EXPORTED_MODELS_DIR}

In [ ]:
health_route = "/v2/health/ready"
predict_route = "/v2/models/deepfm_ens/infer"
serving_container_ports = [8000]
in_container_model_repository = '/models'
serving_container_args = [in_container_model_repository]

model_ensemble_location = os.path.join(EXPORTED_MODELS_DIR, local_exported_ensemble_path)

In [ ]:
model = vertex_ai.Model.upload(
    display_name=model_display_name,
    description=model_description,
    serving_container_image_uri=IMAGE_URI,
    serving_container_predict_route=predict_route,
    serving_container_health_route=health_route,
    serving_container_ports=serving_container_ports,
    artifact_uri=model_ensemble_location,
    serving_container_args=serving_container_args,
    sync=True
)

model.resource_name

## 3. Building a custom container derived from NVIDIA NGC Merlin inference image.

In [ ]:
! gcloud builds submit --timeout "2h" --tag {IMAGE_URI} {DOCKERFILE} --machine-type=e2-highcpu-8

## 4. Deploy the model to Vertex AI Prediction.

### Create the Vertex Endpoint

In [ ]:
vertex_ai.Endpoint.create(
    display_name=ENDPOINT_DISPLAY_NAME
)

### Deploy the model to Vertex Endpoint

In [ ]:
traffic_percentage = 100
machine_type = "n1-standard-8"
accelerator_type="NVIDIA_TESLA_T4"
accelerator_count = 1
min_replica_count = 1
max_replica_count = 3

In [ ]:
model.deploy(
    endpoint=endpoint,
    deployed_model_display_name=MODEL_DISPLAY_NAME,
    machine_type=machine_type,
    min_replica_count=min_replica_count,
    max_replica_count=max_replica_count,
    traffic_percentage=traffic_percentage,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    sync=True,
)

## 5. Getting the inference on a sample data points using hte endpoint.

In [ ]:
from src.serving import inference

data = pd.read_csv('gs://diman-criteo/data/criteo.csv', encoding='utf-8', index_col=[0])

# Defining whether to set the request_data in json or binary format
binary_data = False

data = data[[x for x in data.columns if x != "label"]].fillna(0)

# Converting the data into triton's InferInput object format 
# The format matches KF Serving V2 protocol
inputs = inference.get_inference_input(data, binary_data)

# Greating the request_body to be sent to the inference request  
if (binary_data):
    request_body, json_size = inference.get_inference_request(inputs, '1')
    with open('criteo.dat', 'wb') as output_file:
        output_file.write(request_body)
else:
    infer_request, request_body, json_size = inference.get_inference_request(inputs, '1')
    json_obj = json.loads(request_body)
    with open('criteo.json', 'w') as output_file:
        json.dump(json_obj, output_file)
         
output_file.close()
    

### Getting inference for a json input using curl command

In [ ]:
%%bash -s  $PROJECT_ID $REGION $ENDPOINT_DISPLAY_NAME

PROJECT_ID=$1
REGION=$2
endpoint_display_name=$3

# get endpoint id
echo "REGION = ${REGION}"
echo "ENDPOINT DISPLAY NAME = ${ENDPOINT_DISPLAY_NAME}"
ENDPOINT_ID=$(gcloud beta ai endpoints list --region ${REGION} --filter "display_name=${ENDPOINT_DISPLAY_NAME}" --format "value(ENDPOINT_ID)")
echo "ENDPOINT_ID = ${ENDPOINT_ID}"

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json"  \
  https://${REGION}-aiplatform.googleapis.com/v1/projects/${PROJECT_ID}/locations/${REGION}/endpoints/${ENDPOINT_ID}:rawPredict \
  -d @criteo.json